### example code for submitting a distributed dask job with a function and a list
#### more info on dask distributed client https://distributed.dask.org/en/stable/client.html

## A normal looping of a list

In [1]:
import time
from tqdm.notebook import tqdm

In [2]:
def process_one_item(item):
    # result = some_magic(item)
    time.sleep(1)
    result = 'done'
    return result

In [3]:
list_of_things = list(range(30))

In [4]:
%%time
for i in tqdm(list_of_things):
    process_one_item(i)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 141 ms, sys: 16.6 ms, total: 158 ms
Wall time: 30.2 s


## start a dask cluster

In [4]:
from hyperplane import notebook_common as nc
num_workers = 2

## node specific parameters
total_memory = 12  #110 GB allocatible for 16_128 nodes, 12G for 16_16 nodes, 27G for 32_32
cors_per_worker = 15   # 15 cores for 16_128 nodes and 16_16 nodes, 28 cores for 32_32 nodes
nprocs = 15
ram_gb_per_proc = total_memory/nprocs
nthreads = int(cors_per_worker/nprocs)

print(f'initializing with {num_workers} num_workers, {nprocs} nprocs each proc has {ram_gb_per_proc} GB')
client, cluster = nc.initialize_cluster(
        num_workers = num_workers,
        nprocs = nprocs,
        nthreads = nthreads,
        ram_gb_per_proc = ram_gb_per_proc,
        cores_per_worker = cors_per_worker
    )

initializing with 2 num_workers, 15 nprocs each proc has 0.8 GB
👉 Hyperplane: selecting worker node pool
👉 Hyperplane: selecting scheduler node pool
Creating scheduler pod on cluster. This may take some time.
👉 Hyperplane: spinning up a dask cluster with a scheduler as a standalone container.
👉 Hyperplane: In a few minutes you'll be able to access the dashboard at https://riskthinking1.hyperplane.dev/dask-cluster-6b30e880-cf65-4250-b052-39eabdeda3e9/status
👉 Hyperplane: to get logs from all workers, do `cluster.get_logs()`


In [4]:
## whenever you want to clean up the dask memory this is the magic line
client.restart()

Client Scheduler: tcp://dask-cluster-dd70f1bf-30ef-4ab3-ae97-46f2d3d1404e.jhub-5hjo3sdt:8786 Dashboard: http://dask-cluster-dd70f1bf-30ef-4ab3-ae97-46f2d3d1404e.jhub-5hjo3sdt:8787/status,Cluster Workers: 30 Cores: 30 Memory: 22.35 GiB


## use dask bags

In [6]:
len(list_of_things)

30

In [7]:
%%time 
from dask import bag as db
bag_list = db.from_sequence(list_of_things, npartitions=len(client.nthreads()))
results = db.map(process_one_item, bag_list).compute()
print(len(results), set(results))

30 {'done'}
CPU times: user 24.4 ms, sys: 4.15 ms, total: 28.6 ms
Wall time: 1.22 s


## use barebone dask distributed

In [8]:
%%time
## run it on dask cluster for all urls
L = client.map(process_one_item, list_of_things)

CPU times: user 3.1 ms, sys: 116 µs, total: 3.21 ms
Wall time: 2.41 ms


In [12]:
%%time
results_distributed = client.gather(L)
print(results_distributed) ## this will display a list of result, results being the return from process_one_url function

['done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done', 'done']
CPU times: user 5.31 ms, sys: 1.04 ms, total: 6.35 ms
Wall time: 15.7 ms


In [13]:
client.close()
cluster.close()